In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cycler
import seaborn as sns

height = 3.7
aspect = 6/height

plot_line_cycle = cycler('linestyle', ['-', '--', ':', '-.'])
plt.rcParams['figure.dpi'] = 120
params = {"figure.figsize": [6,3.7],
            "font.size": 14,
            "legend.handlelength": 1.4,
            "legend.fontsize": 11,
            "text.usetex": True,
            "axes.labelpad": 10,
            "axes.linewidth": 1.1,
            "axes.xmargin": 0.05,
            "axes.prop_cycle": plot_line_cycle,
            "axes.autolimit_mode"  : "data",
            "xtick.major.size"     : 7,
            "xtick.minor.size"     : 3.5,
            "xtick.major.width"    : 1.1,
            "xtick.minor.width"    : 1.1,
            "xtick.major.pad"      : 5,
            "xtick.minor.visible"  : True,
            "ytick.major.size"     : 7,
            "ytick.minor.size"     : 3.5,
            "ytick.major.width"    : 1.1,
            "ytick.minor.width"    : 1.1,
            "ytick.major.pad"      : 5,
            "ytick.minor.visible"  : True,
            "lines.markersize"     : 7,
            "lines.markerfacecolor" : "none",
            "lines.markeredgewidth"  : 0.8}

params["figure.figsize"] = [6,4]
params["font.family"] =  'serif'
params["font.sans-serif"] = 'cm'
params["axes.linewidth"] = 1.3
params["legend.title_fontsize"] = 11
plt.rcParams.update(params)

In [ ]:
df_pre = pd.read_csv('data/sf_clean.csv', sep=',')
df = df_pre.loc[(df_pre['patient'] < 31)]
adrenal = df.loc[(df['target']=='adrenal_glands')]
adrenal_main = adrenal.loc[adrenal['limiting']==1]
adrenal_duo = adrenal_main.loc[(adrenal_main['oar']=='duodenum')]
pancreas = df.loc[(df['target']=='pancreas')]
prostate = df.loc[(df['target']=='boost') | (df['target']=='prostate')]
boost = df.loc[(df['target']=='boost')]
# df.columns

In [ ]:
def plot_single(data, x, y, hue, x_label=None, y_label=None, minor_ticks=True, palette='Set2'):
    ax = sns.scatterplot(data=data, x=x, y=y, hue=hue, palette=palette, alpha=.8)
    if not minor_ticks:
        ax.tick_params(axis='x', which='minor', bottom=False)
    fig = ax.get_figure()
    if x_label != None and y_label != None:
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
    ax.legend_.set_title('OAR')
    fig.subplots_adjust(left=0.18, bottom=0.2, right=0.93, top=0.95)
    return fig

def plot_grid(data, x, y, hue, row, x_label=None, y_label=None, palette='Set2'):
    fig = sns.FacetGrid(data=data, hue=hue, row=row, height=height, aspect=aspect, palette=palette)
    fig.map(sns.scatterplot, x, y, alpha=.8)
    fig.despine(top=False, right=False)
    fig.add_legend()
    fig.legend.set_title('Patient')
    if x_label != None and y_label != None:
        fig.set_axis_labels(x_label, y_label)
    fig.set_titles("OAR: {row_name}")
    fig.tight_layout(w_pad=1)
    return fig

def plot_single_fraction(data, x, y, hue, x_label, y_label, y_twin_label, y_twin=None, palette='Set2'):
    fig, ax = plt.subplots()
    ax = sns.scatterplot(data=data, x=x, y=y, hue=hue, palette=palette, marker='^')
    if y_twin != None:
        ax2 = ax.twinx()
        sns.lineplot(data=data, x=x, y=y_twin, hue=hue, palette=palette, linestyle='-')
        ax2.get_legend().remove()
        ax2.invert_yaxis()
        ax2.set_ylabel(y_twin)
        ax2.set_ylabel(y_twin_label)
    ax.set(xlabel=x_label, ylabel=y_label)
    ax.legend_.set_title('Patient')
    return fig

def plot_twin_grid(data, x, y, y_twin, hue, row, x_label=None, y_label=None, y_twin_label=None, palette='Set2'):
    palette_dict = dict(zip(data[hue].unique(), sns.color_palette(palette, n_colors=len(data[hue].unique()))))

    fig = sns.relplot(data=data, x=x, y=y, hue=hue, row=row, palette=palette_dict, style=hue, markers=['^'], height=height, aspect=aspect)
    for index, [oar, ax] in enumerate(fig.axes_dict.items()):
        ax2 = ax.twinx()
        data_sub = data.loc[data[row]==oar]
        sns.lineplot(data=data_sub, x=x, y=y_twin, hue=hue, palette=palette_dict, linestyle='-')
        ax2.get_legend().remove()
        ax2.invert_yaxis()
        ax2.set_ylabel(y_twin_label)
    fig.set_axis_labels(x_label, y_label)
    fig.legend.set_title('Patient')
    fig.set_titles("OAR: {row_name} organ")
    fig.tick_params(axis='x', which='minor', bottom=False)
    fig.tight_layout(w_pad=1)
    return fig

In [ ]:
adrenal_single_d_sf = plot_single(adrenal, 'distance', 'sparing_factor', 'oar', r'$w$ [cm]', r'$\delta$')

In [ ]:
adrenal_single_pat_sf = plot_single(adrenal_main, 'patient', 'sparing_factor', 'oar', 'Patient', r'$\delta$', minor_ticks=False)

In [ ]:
adrenal_grid_d_sf = plot_grid(adrenal, 'distance', 'sparing_factor', 'patient', 'oar', r'$w$ [cm]', r'$\delta$')

In [ ]:
adrenal_grid_f_sf = plot_twin_grid(adrenal_main, 'fraction', 'sparing_factor', 'distance', 'patient', 'oar', r'Fraction $t$', r'$\delta$', r'$w$ [cm]')

In [ ]:
adrenal_grid_f_sf_dose = plot_twin_grid(adrenal_main, 'fraction', 'sparing_factor', 'd_T', 'patient', 'oar', r'Fraction $t$', r'$\delta$', '$d_T$ [Gy]')

In [ ]:
def save_plot(fig, basename):
    fig.savefig(f'{basename}.pdf', format='pdf')
save_plot(adrenal_single_d_sf, '1')
save_plot(adrenal_single_pat_sf, '2')
save_plot(adrenal_grid_d_sf, '3')
save_plot(adrenal_grid_f_sf, '4')
save_plot(adrenal_grid_f_sf_dose, '5')

In [ ]:
figurine = plot_single_fraction(prostate, 'fraction', 'sparing_factor', 'patient', r'Fraction $t$', r'$\delta$', r'$w$ [cm]', 'distance')

In [ ]:
g = sns.FacetGrid(df, hue="oar", col="target")
g.map(sns.scatterplot, 'distance', 'sparing_factor', alpha=.7)
g.despine(top=False, right=False)
g.add_legend()

In [ ]:
pancreas_single_d_sf = plot_single(pancreas, 'distance', 'sparing_factor', 'oar', r'$w$ [cm]', r'$\delta$')

In [ ]:
adrenal_single_pat_sf = plot_single(pancreas, 'patient', 'sparing_factor', 'oar', 'Patient', r'$\delta$', minor_ticks=False)

In [ ]:
g = sns.FacetGrid(pancreas, hue="patient", col="oar")
g.map(sns.scatterplot, 'distance', 'sparing_factor', alpha=.7)
g.despine(top=False, right=False)
g.add_legend()
g.set_axis_labels(r"$d$ [cm]", r"$\delta$")
g.set_titles("OAR: {col_name} organ")
g.tight_layout(w_pad=1)

In [ ]:
sns.scatterplot(data=prostate, x='distance', y='sparing_factor', hue='target', palette='Set2')

In [ ]:
sns.histplot(df_pre.loc[(df_pre['target']!='prostate')], x='sparing_factor', hue='target', bins=13, element='step')